# NILMTK Rapid Experimentation API

### This notebook demonstrates the use of NILMTK's ExperimentAPI - a  new  NILMTK  interface which  allows NILMTK users to focus on which experiments to run rather than on the code required to run such experiments.

It is important to note that handing over so much flexibility to the user does require the user to be somewhat familiar with the data set, but this part of the process is supported by NILMTK as data exploration is simple and well documented. 

Lets us start with a very simple experiment to demonstrate the use of the API for multiple appliances in a minimal use case. This experiment shows how the user can select the appliances in the dataset on which disaggregation is to be performed.

Importing the API. 

In [1]:
from nilmtk.api import API
import warnings
warnings.filterwarnings("ignore")

Next, we import the required algorithms on which we wish to run the experiments

In [2]:
from nilmtk.disaggregate import CO

Next, we enter the values for the different parameters in the dictionary. Since we need multiple appliances, we enter the names of all the required appliances in the _'appliances'_ parameter.

In [3]:
experiment1 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['fridge','air conditioner', 'microwave'],
  'methods': {"CO":CO({})},
  'train': {    
    'datasets': {
        'Dataport': {
            'path': 'data/dataport.hdf5',
            'buildings': {
                10: {
                    'start_time': '2015-04-04',
                    'end_time': '2015-04-06'
                    }
                }                
            }
        }
    },
  'test': {
    'datasets': {
        'Dataport': {
            'path': 'data/dataport.hdf5',
            'buildings': {
                10: {
                    'start_time': '2015-04-25',
                    'end_time': '2015-04-26'
                    }
                }
            }
        },
        'metrics':['rmse']
    }
}

In this example experimental setup, we have set the _sample rate_ at 60Hz and use Combinatorial Optimisation to 
disaggregate the required appliances from building 10 in the dataport dataset with the _RMSE_ metric to measure the accuracy. We also specify the dates for training and testing

Next we provide this experiment dictionary as input to the API.

In [4]:
api_results_experiment_1 = API(experiment1)

Joint Testing for all algorithms
Loading data for  Dataport  dataset
Dropping missing values
Generating predictions for : CO
...............CO disaggregate_chunk running.............
............  rmse  ..............wave'ioner'
                         CO
fridge            92.925652
air conditioner  487.541665
microwave        746.574343


We can observe the prediction vs. truth graphs in the above cell. The accuracy metrics can be accessed using the following commands:

In [5]:
errors_keys = api_results_experiment_1.errors_keys
errors = api_results_experiment_1.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

Dataport_10_rmse
                         CO
fridge            92.925652
air conditioner  487.541665
microwave        746.574343





This was a trivial experiment that only scratches the surface of the true potential of this API.

In the next experiment we will run an incrementally more complex version of the above experiment. Here we will use multiple models to disaggregate the appliance readings with the models having their own sets of parameters which can be set by the users within the experimental dictionary in order to fine tune experiments.

We also import the required algorithms for the next experiments

In [6]:
from nilmtk.disaggregate import FHMM_EXACT, Mean

In [7]:
experiment2 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['fridge','air conditioner', 'microwave'],
  'methods': {"Mean":Mean({}),"FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), "CombinatorialOptimisation":CO({})},
  'train': {    
    'datasets': {
        'Dataport': {
            'path': 'data/dataport.hdf5',
            'buildings': {
                10: {
                    'start_time': '2015-04-04',
                    'end_time': '2015-04-06'
                    }
                }                
            }
        }
    },
  'test': {
    'datasets': {
        'Datport': {
            'path': 'data/dataport.hdf5',
            'buildings': {
                10: {
                    'start_time': '2015-04-25',
                    'end_time': '2015-04-26'
                    }
                }
            }
        },
        'metrics':['mae', 'rmse']
    }
}

In [8]:
api_results_experiment_2 = API(experiment2)

Joint Testing for all algorithms
Loading data for  Datport  dataset
Dropping missing values
Generating predictions for : Mean
Generating predictions for : FHMM
Generating predictions for : CO
...............CO disaggregate_chunk running.............
............  mae  ..............owave'ioner'
                       Mean  FHMM_EXACT  CombinatorialOptimisation
fridge            45.440582   77.579147                  79.816048
air conditioner  397.251617  370.581329                 274.858612
microwave         36.396835  436.638092                 461.106995
............  rmse  ..............
                       Mean  FHMM_EXACT  CombinatorialOptimisation
fridge            60.877859   93.722053                  93.047745
air conditioner  607.580550  575.861339                 487.461264
microwave        166.414246  668.473027                 746.621181


In [9]:
api_results_experiment_2.errors

[                       Mean  FHMM_EXACT  CombinatorialOptimisation
 fridge            45.440582   77.579147                  79.816048
 air conditioner  397.251617  370.581329                 274.858612
 microwave         36.396835  436.638092                 461.106995,
                        Mean  FHMM_EXACT  CombinatorialOptimisation
 fridge            60.877859   93.722053                  93.047745
 air conditioner  607.580550  575.861339                 487.461264
 microwave        166.414246  668.473027                 746.621181]

In [10]:
errors_keys = api_results_experiment_2.errors_keys
errors = api_results_experiment_2.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

Datport_10_mae
                       Mean  FHMM_EXACT  CombinatorialOptimisation
fridge            45.440582   77.579147                  79.816048
air conditioner  397.251617  370.581329                 274.858612
microwave         36.396835  436.638092                 461.106995



Datport_10_rmse
                       Mean  FHMM_EXACT  CombinatorialOptimisation
fridge            60.877859   93.722053                  93.047745
air conditioner  607.580550  575.861339                 487.461264
microwave        166.414246  668.473027                 746.621181





The API makes running experiments extremely quick and efficient, with the emphasis on creating finely tuned reproducible experiments where model and parameter performances can be easily evaluated at a glance.   

In the next iteration of this experiment, we introduce more parameters _chunksize_, _DROP_ALL_NANS_ and _artificial_aggregate_ and add another disaggregation algorithm (_Hart85_). We also train and test data from multiple buildings of the same dataset.

We also import the Hart algorithm for the next experiment

In [11]:
from nilmtk.disaggregate import Hart85

In [13]:
experiment3 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['fridge','air conditioner','electric furnace','washing machine'],
  'artificial_aggregate': True,
  'chunksize': 20000,
  'DROP_ALL_NANS': False,
  'methods': {"Mean":Mean({}),"Hart85":Hart85({}), "FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), "CO":CO({})},
  'train': {    
    'datasets': {
      'Dataport': {
        'path': 'data/dataport.hdf5',
        'buildings': {
          54: {
            'start_time': '2015-01-28',
            'end_time': '2015-02-12'
          },
          56: {
            'start_time': '2015-01-28',
            'end_time': '2015-02-12'
          },
          57: {
            'start_time': '2015-04-30',
            'end_time': '2015-05-14'
          },
                }                
            }
        }
    },
  'test': {
    'datasets': {
      'Datport': {
        'path': 'data/dataport.hdf5',
        'buildings': {
          94: {
            'start_time': '2015-04-30',
            'end_time': '2015-05-07'
          },
          103: {
            'start_time': '2014-01-26',
            'end_time': '2014-02-03'
          },
          113: {
            'start_time': '2015-04-30',
            'end_time': '2015-05-07'
          },
        }
      }
    },
        'metrics':['mae', 'rmse']
    }
}

In [14]:
api_results_experiment_3 = API(experiment3)

Joint Testing for all algorithms
Loading data for  Datport  dataset
Dropping missing values
Creating an Artificial Aggregate
Generating predictions for : Mean
Generating predictions for : Hart85
Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.
Generating predictions for : FHMM
Generating predictions for : CO
...............CO disaggregate_chunk running.............
............ wer demand for 'washing machine'' mae  ..............
                        Mean     Hart85  FHMM_EXACT         CO
fridge             60.475124  26.411211   16.998512  58.242363
air conditioner   133.285614   5.055853   22.030258  72.022423
electric furnace   63.302246  88.532242   25.158630  10.183631
washing machine     2.734084  82.629463    1.800099  12.294147
............  rmse  ..............
                        Mean      Hart85  FHMM_EXACT          CO
fridge             75.940336 

In [15]:
errors_keys = api_results_experiment_3.errors_keys
errors = api_results_experiment_3.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

Datport_94_mae
                        Mean     Hart85  FHMM_EXACT         CO
fridge             60.475124  26.411211   16.998512  58.242363
air conditioner   133.285614   5.055853   22.030258  72.022423
electric furnace   63.302246  88.532242   25.158630  10.183631
washing machine     2.734084  82.629463    1.800099  12.294147



Datport_94_rmse
                        Mean      Hart85  FHMM_EXACT          CO
fridge             75.940336   53.922593   63.108259   87.623677
air conditioner   204.703034   40.421356  123.048184  187.298225
electric furnace   82.090731  117.664927   93.185257   21.753245
washing machine    10.896916  107.468327   11.097525   51.306552



Datport_103_mae
                        Mean     Hart85  FHMM_EXACT         CO
fridge             64.257904   3.944097    8.862674  45.464237
air conditioner   132.515869   5.799566   29.786198  35.921093
electric furnace   95.431465  74.070572   27.676476  45.702343
washing machine     2.847503  29.716927    1.721615  21

The results of the above experiment are presented for every chunk per building in the test set.

In the following experiment, we demonstrate how to run experiments across datasets, which was previously not possible. The important thing to pay attention to is that such datasets can only be trained and tested together as long as they have common appliances in homes with common _ac_types_.

In [16]:
experiment4 = {
  'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['washing machine','fridge'],
  'artificial_aggregate': True,
  'chunksize': 20000,
  'DROP_ALL_NANS': False,
  'methods': {"Mean":Mean({}),"Hart85":Hart85({}), "FHMM_EXACT":FHMM_EXACT({'num_of_states':2}), 'CO':CO({})},   
  'train': {
    'datasets': {
      'UKDALE': {
        'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/ukdale.h5',
        'buildings': {
              1: {
                'start_time': '2017-01-05',
                'end_time': '2017-03-05'
              },          
            }
          },        
        }
      },    
  'test': {
    'datasets': {
      'DRED': {
        'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/dred.h5',
        'buildings': {
              1: {
                    'start_time': '2015-09-21',
                    'end_time': '2015-10-01'
          }
        }
      },
      'REDD': {
        'path': 'C:/Users/Hp/Desktop/nilmtk-contrib/redd.h5',
        'buildings': {
              1: {
                    'start_time': '2011-04-17',
                    'end_time': '2011-04-27'
          }
        }
      }
    },
        'metrics':['mae', 'rmse']
    }
}

In [17]:
api_results_experiment_4 = API(experiment4)

Joint Testing for all algorithms
Loading data for  DRED  dataset
Dropping missing values
Creating an Artificial Aggregate
Generating predictions for : Mean
Generating predictions for : Hart85
Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.
Generating predictions for : FHMM
Generating predictions for : CO
...............CO disaggregate_chunk running.............
............  mae  ..............ge' machine'
                      Mean    Hart85  FHMM_EXACT         CO
washing machine  25.636555  2.186584    2.276629  24.139162
fridge           43.427475  7.185967    4.252556  25.250912
............  rmse  ..............
                      Mean     Hart85  FHMM_EXACT         CO
washing machine  54.964733  29.918196   31.821257  50.603907
fridge           45.180869  20.587223   10.595766  49.527368
Loading data for  REDD  dataset
Loading data for meter ElecMeterID(ins

In [18]:
errors_keys = api_results_experiment_4.errors_keys
errors = api_results_experiment_4.errors
for i in range(len(errors)):
    print (errors_keys[i])
    print (errors[i])
    print ("\n\n")

DRED_1_mae
                      Mean    Hart85  FHMM_EXACT         CO
washing machine  25.636555  2.186584    2.276629  24.139162
fridge           43.427475  7.185967    4.252556  25.250912



DRED_1_rmse
                      Mean     Hart85  FHMM_EXACT         CO
washing machine  54.964733  29.918196   31.821257  50.603907
fridge           45.180869  20.587223   10.595766  49.527368



REDD_1_mae
                      Mean     Hart85  FHMM_EXACT         CO
washing machine  47.244759   9.858242   20.557049  28.607571
fridge           62.311844  53.618702   32.754597  32.168388



REDD_1_rmse
                       Mean      Hart85  FHMM_EXACT         CO
washing machine  236.715613  114.865967  186.791693  78.342796
fridge            86.095885  100.099604   61.811505  56.396517





Just like the above experiments, any user can set up other experiments very quickly. 